In [21]:
# @langchain_api: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

We create a prompt that explicitly instructs the LLM to output a structured sequence of Thought, Action, and Action Input.


A. Defining the Agent State (The Shared Memory)

In [ ]:

import operator
from typing import Annotated, List, TypedDict, Optional
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, ToolMessage

# --- AgentState Definition ---
class AgentState(TypedDict):
    """
    Represents the state of the agent's work. It is the single source of truth
    passed between all nodes in the graph.
    """
    # 1. Conversation History (Reducer: Append messages)
    messages: Annotated[List[BaseMessage], operator.add] 
    '''Key Concept: We use Annotated with operator.add for the messages list. 
    This is the reducer function that tells LangGraph: 
    "When a node returns new messages, append them to the existing list, don't overwrite the history.
    "'''
    
    # 2. Tool Action (Used by the Conditional Edge/Tool Executor)
    # The name of the tool the agent decided to use (e.g., 'search_tool')
    action_name: Optional[str] 
    
    # 3. Tool Input (The arguments for the tool)
    # The JSON string or dictionary passed to the tool
    action_input: Optional[dict]

B. ReAct Prompt Template

Since we are using watsonx.ai models that don't natively support tool calling, we must use a system prompt to enforce the ReAct structure.

In [3]:
REACT_PROMPT = """
You are a specialized agent. Your goal is to answer the user's request.
You have access to the following tool: {tool_name} with the following description: {tool_description}

You must respond in one of two formats:

1. Final Answer:
Thought: I have enough information to answer the user.
Action: Final Answer
Action Input: The final answer goes here.

2. Tool Call:
Thought: I need to use the tool to find the answer.
Action: {tool_name}
Action Input: {{"query": "the search term goes here"}}

Begin.
"""

In [7]:
from ibm_watsonx_ai import APIClient, Credentials
from ibm_watsonx_ai.foundation_models import ModelInference
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from typing import Any

In [9]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [11]:
def get_watsonx_llm(
        model_id="meta-llama/llama-3-2-11b-instruct",
        max_new_tokens=300,
        temperature=0.2
    ) -> ModelInference:

    creds = Credentials(
        api_key = os.getenv("WATSONX_API_KEY"),
        url = os.getenv("WATSONX_URL") 
    )

    client = APIClient(credentials=creds)

    return ModelInference(
        model_id=model_id,
        api_client=client,
        project_id=os.getenv("WATSONX_PROJECT_ID"),
        params={
            GenParams.MAX_NEW_TOKENS: max_new_tokens,
            GenParams.TEMPERATURE: temperature,
        }
    )


In [22]:
def react_llm_node(state: AgentState, tool_name: str, tool_description: str):
    """
    LangGraph node that:
    1. Builds the ReAct prompt using conversation history
    2. Sends it to Watsonx LLaMA (ModelInference)
    3. Produces an AIMessage containing the model output
    4. Updates the 'messages' list in AgentState
    """
    
    # ---- 1. Build the prompt by combining all previous messages ----
    conversation = ""
    for msg in state["messages"]:
        if isinstance(msg, HumanMessage):
            conversation += f"User: {msg.content}\n"
        elif isinstance(msg, AIMessage):
            conversation += f"Assistant: {msg.content}\n"
        elif isinstance(msg, ToolMessage):
            conversation += f"Tool: {msg.content}\n"

    # Include your ReAct base template
    prompt = REACT_PROMPT.format(
        tool_name=tool_name,
        tool_description=tool_description
    ) + "\n" + conversation + "\n"

    # ---- 2. Run the LLaMA model through Watsonx ----
    llm = get_watsonx_llm()

    response = llm.generate_text(prompt)
    raw_text = response["results"][0]["generated_text"]

    # ---- 3. Wrap into AIMessage ----
    ai_msg = AIMessage(content=raw_text)

    # ---- 4. Return update for LangGraph state ----
    return {"messages": [ai_msg]}


C. The ReAct Parsing Node (The Crucial Step)

This function takes the raw text output from the watsonx.ai model and uses regular expressions to extract the Action and Action Input needed for the graph logic.

In [4]:
import re

def parse_and_decide(state: AgentState) -> AgentState:
    """
    Parses the last AI message for ReAct structure and updates the state.
    This also acts as the router for the conditional edge.
    """
    last_message = state["messages"][-1]
    
    # Ensure we are parsing an AI message
    if not isinstance(last_message, AIMessage):
        return {} # No update if it's not the LLM's output

    text = last_message.content
    print(f"\n--- Parsing LLM Output: {text[:50]}... ---")

    # Regex to find the Action and Action Input
    action_match = re.search(r"Action: (.+?)\nAction Input: (.+)", text, re.DOTALL)

    if action_match:
        action_name = action_match.group(1).strip()
        action_input_str = action_match.group(2).strip()
        
        # Try to parse Action Input as JSON
        try:
            # We assume Action Input is a simple JSON string {"query": "..."}
            action_input = eval(action_input_str) 
        except:
            print("Warning: Could not parse Action Input. Assuming final answer.")
            action_name = "Final Answer"
            action_input = None

        if action_name == "Final Answer":
            # The agent decided to finish. Clear action tracking fields.
            return {"action_name": None, "action_input": None}
        else:
            # The agent decided to use a tool. Update state for the next node.
            return {"action_name": action_name, "action_input": action_input}
    
    # If no clear ReAct structure is found, assume the LLM returned a final answer.
    return {"action_name": None, "action_input": None}

D. The Conditional Edge Logic (The Router)

The router is now trivial because the parse_and_decide node already updated the state with the action name.

In [5]:
def should_continue(state: AgentState) -> str:
    """Checks the state to see if a tool call is pending or if the agent is done."""
    
    # If action_name is set by the parser, we need to call the tool
    if state.get("action_name"):
        return "call_tool"
    else:
        # Otherwise, the parser decided the LLM gave the final answer
        return "end"

In [16]:
def tool_executor_node(state: AgentState, tools: dict):
    tool_name = state["action_name"]
    action_input = state["action_input"]

    tool_fn = tools.get(tool_name)
    result = tool_fn(**action_input)

    tool_msg = ToolMessage(content=str(result), name=tool_name)
    return {"messages": [tool_msg], "action_name": None, "action_input": None}


In [19]:
def route(state: AgentState):
    if state["action_name"] is None:
        return "finish"
    return "tool_executor"


In [20]:
from langgraph.graph import StateGraph, START, END

graph = StateGraph(AgentState)

# Nodes
graph.add_node(
    "react_llm",
    lambda state: react_llm_node(
        state,
        tool_name="search_tool",
        tool_description="Searches indexed documents."
    )
)

graph.add_node("router", parse_and_decide)
graph.add_node("tool_executor", tool_executor_node)

# Edges
graph.add_edge(START, "react_llm")
graph.add_edge("react_llm", "router")

def route(state: AgentState):
    return "end" if state["action_name"] is None else "tool"

graph.add_conditional_edges(
    "router",
    route,
    {
        "end": END,
        "tool": "tool_executor"
    }
)

graph.add_edge("tool_executor", "react_llm")

# Compile
app = graph.compile()
